**TODO:**

* When merging cells, merge their contents.
* Implement data export.
* Table-aware plot embedding.
    * It should be possible to embed a bar plot or a line plot simply by assigning a special "formatter" to a row / column.
    * The layout of the embedded plot should be responsive to cell spacing, gaps, padding, etc.
    * The tricky part will be handling formatters that get overridden / are discontiguous.

In [1]:
import numpy
import toyplot.data

import logging
toyplot.log.setLevel(logging.DEBUG)

In [10]:
data = numpy.arange(25).reshape(5, 5)
canvas, table = toyplot.table(data, width=500, height=500)
table.cells.grid.hlines[...] = "single"
table.cells.grid.vlines[...] = "single"
#table.body.row[2].delete()
#table.body.column[2].delete()
table.body.row.insert(before=2)

DEBUG:toyplot:insert 0: [False False  True False False False False]


0 1 2 3 4 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24

In [3]:
data = toyplot.data.Table()
data["a"] = numpy.array(["Foo-BAZ-46", "Bar-BAZ-46", "Foo-BAZ-23", "Bar-BAZ-23", "Foo-BLAH", "Bar-BLAH"])
data["b"] = numpy.array([0.381, 0.142, 0.473, 0.234, 0.165, 0.186])

In [4]:
canvas, table = toyplot.table(data, width=600, height=250)
table.cells.grid.hlines[...] = "single"
table.cells.grid.vlines[...] = "single"
#table.cells.column[0].lstyle = {"-toyplot-anchor-shift":"0.75in"}
table.cells.column[0].align = "center"

a b Foo-BAZ-46 0 . 381 Bar-BAZ-46 0 . 142 Foo-BAZ-23 0 . 473 Bar-BAZ-23 0 . 234 Foo-BLAH 0 . 165 Bar-BLAH 0 . 186

In [5]:
colormap = toyplot.color.CategoricalMap()

canvas = toyplot.Canvas(width=600, height=250)
table = canvas.table(rows=6, columns=4, label="Quarterly Report")
table.cells.grid.hlines[...] = "single"
table.cells.grid.vlines[...] = "single"
table.cells.column[0].width = 50
table.cells.column[1].data = data["a"]
table.cells.column[1].width = 90
table.cells.column[3].data = data["b"]
table.cells.column[3].width = 55
table.cells.column[3].format = toyplot.format.FloatFormatter(format="{:.2f}s")
table.cells.cell[0:2, 0].merge().data = "Red"
table.cells.cell[2:4, 0].merge().data = "Green"
table.cells.cell[4:6, 0].merge().data = "Blue"
axes = table.cells.cell[0:6, 2].cartesian(show=True, yshow=False, padding=0, xlabel="Seconds")
axes.x.spine.show = False
axes.bars(data["b"][::-1], along="y", style={"stroke-width":8}, color=(numpy.array([0,1,0,1,0,1])[::-1], colormap));

Quarterly Report Foo-BAZ-46 0.38s Bar-BAZ-46 0.14s Foo-BAZ-23 0.47s Bar-BAZ-23 0.23s Foo-BLAH 0.17s Bar-BLAH 0.19s Red Green Blue 0.0 0.1 0.2 0.3 0.4 0.5 Seconds 
 
 
 Save as .csv

In [6]:
canvas = toyplot.Canvas(width=600, height=250)
table = canvas.table(rows=6, columns=4, label="Quarterly Report")
table.cells.column[0].width = 50
table.cells.column[1].data = data["a"]
table.cells.column[1].width = 90
table.cells.column[3].width = 55
table.cells.column[3].format = toyplot.format.FloatFormatter(format="{:.2f}s")
table.cells.cell[0:2, 0].merge().data = "Red"
table.cells.cell[2:4, 0].merge().data = "Green"
table.cells.cell[4:6, 0].merge().data = "Blue"
for index, value in enumerate(data["b"]):
    axes = table.cells.cell[index, 2].cartesian(padding=5, xmin=0, xmax=data["b"].max(), xlabel="Seconds", yshow=False)
    axes.bars([value], along="y", color=colormap.color(index % 2))
    axes.show = index == 5
    axes.x.spine.show = False

Quarterly Report Foo-BAZ-46 Bar-BAZ-46 Foo-BAZ-23 Bar-BAZ-23 Foo-BLAH Bar-BLAH Red Green Blue 0.0 0.1 0.2 0.3 0.4 0.5 Seconds 
 
 
 Save as .csv